1 Slowfashion
1. Specific Category Recognition (deepest subcategories in our category tree)
* Below you’ll find our priority of subcategories to be identified by the
tool (not sure how many you estimate the students can cover so I
added our top 9):
1. Dresses  
2. High Heels  
3. Shoulder Bags  
4. Skirts  
5. Tote Bags  
6. Clutches  
7. Outerwear  
8. Boots  
9. Flats

2 These are all found at the farest end of the group category
of “Women” (Women>Clothing>Dresses and for shoes it’s
Women>Shoes>High Heels and for bags it’s Women>Bags>Shoulder
bags.
2. Grouping (Bags & Shoes):
• For products in categories that all “look characteristic for its cat-
egory” – in our case Bags and Shoes – the AI could identify the
group category, say “Bags”, when unsure of it’s a specific subcate-
gory (e.g. “Should Bag”)
• For example, if the tool cannot with certainty say that the image
resemble a Shoulder Bag, but still make out that it is a Bag, it would
be very useful to have the AI tool identify that the image is most
likely some sort of Bag and give the user a nudge taking them to
the category Bags (where the user themselves can choose the correct
subcategory under Bags). Same for Shoes.
Ideally we as a company could later follow a similar process as the students used
and further work on adding additional subcategories.
1


In [8]:
import pandas as pd
pd.set_option('display.max_colwidth', None) 

In [ ]:
def map_deepfashion_category(df):
    mapping = {
        "short sleeve dress": "Dresses",
        "long sleeve dress": "Dresses",
        "vest dress": "Dresses",
        "sling dress": "Dresses",
        "skirt": "Skirts",
        "long sleeve outwear": "Outerwear",
        "short sleeve outwear": "Outerwear",
    }
    return df[df['category_name'].isin(mapping.keys())].assign(
        custom_label=lambda x: x['category_name'].map(mapping)
    )

In [3]:
information_dataframe = pd.read_csv("../slowfashion/information.csv")
information_dataframe["name"] = information_dataframe["path"].apply(lambda item: item.split("/")[-1].split(".")[0])

In [25]:
information_dataframe.groupby("type").size()

type
BACK              304
FIRST DETAIL      284
FRONT             535
LABEL             304
OTHER            1374
SECOND DETAIL     207
dtype: int64

In [19]:
unique_values = information_dataframe.groupby(["type","name"]).size().sort_values().reset_index()
unique_values.columns = ["type", "name", "total_count"]

In [20]:
unique_values.sort_values(by="total_count", ascending=False)[:10]

,type,name,total_count
2212,BACK,unknown-unknown-unknown-unknown-unknown-back,53
2213,FRONT,unknown-unknown-unknown-unknown-unknown-front,53
2211,LABEL,unknown-unknown-unknown-unknown-unknown-label,52
2210,FIRST DETAIL,unknown-unknown-unknown-unknown-unknown-first-detail,51
2209,SECOND DETAIL,unknown-unknown-unknown-unknown-unknown-second-detail,41
2208,BACK,unknown-womens-fashion-unknown-unknown-unknown-back,31
2207,FRONT,unknown-womens-fashion-unknown-unknown-unknown-front,31
2206,LABEL,unknown-womens-fashion-unknown-unknown-unknown-label,31
2204,OTHER,unknown-unknown-unknown-unknown-unknown-other-0,30
2205,FIRST DETAIL,unknown-womens-fashion-unknown-unknown-unknown-first-detail,30


In [ ]:
information_dataframe

In [24]:
"jopis_jp__ojipos_joipa".count("_")

4

In [21]:
unique_values.sort_values(by="total_count", ascending=False)[10:20]

,type,name,total_count
2203,SECOND DETAIL,unknown-womens-fashion-unknown-unknown-unknown-second-detail,16
2202,OTHER,unknown-unknown-unknown-unknown-unknown-other-1,15
2197,FRONT,christian_dior_mens_fashion_accessories_ties___pocket_squares_standard_0,6
2201,OTHER,christian_dior_mens_fashion_accessories_ties___pocket_squares_standard_1,6
2200,OTHER,christian_dior_mens_fashion_accessories_ties___pocket_squares_standard_2,6
2199,OTHER,christian_dior_mens_fashion_accessories_ties___pocket_squares_standard_3,6
2198,OTHER,unknown-unknown-unknown-unknown-unknown-other-2,6
2196,LABEL,redesign-by-emmy-womens-fashion-jewellery-hair-accessories-m-label,6
2195,FRONT,redesign-by-emmy-womens-fashion-jewellery-hair-accessories-m-front,5
2194,FRONT,pierre_cardin_mens_fashion_accessories_ties___pocket_squares_standard_0,5


In [11]:

information_dataframe.head()

,type,path,name
0,FRONT,../slowfashion/FRONT/unknown-womens-fashion-unknown-unknown-unknown-front.png,unknown-womens-fashion-unknown-unknown-unknown-front
1,FRONT,../slowfashion/FRONT/acne-studios-womens-fashion-clothing-tops-t-shirts-m-front.png,acne-studios-womens-fashion-clothing-tops-t-shirts-m-front
2,FRONT,../slowfashion/FRONT/acne-studios-womens-fashion-shoes-high-heels-39-front.png,acne-studios-womens-fashion-shoes-high-heels-39-front
3,FRONT,../slowfashion/FRONT/saint-laurent-womens-fashion-shoes-high-heels-38-front.png,saint-laurent-womens-fashion-shoes-high-heels-38-front
4,FRONT,../slowfashion/FRONT/gucci-womens-fashion-shoes-sneakers-unknown-front.png,gucci-womens-fashion-shoes-sneakers-unknown-front
